In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
# from linearmodels.panel.data import PanelData
# from linearmodels.panel import PanelOLS, PooledOLS, RandomEffects, compare
import matplotlib.pyplot as plt
import pyblp

In [3]:
product_data=pd.read_csv("../data/merged/len3_ndb_agg_blp_imputed_lags.csv",encoding="shift-jis",index_col=0)
product_data["market_ids"]=product_data["薬効分類"].astype(int).astype(str)+"-"+product_data["year"].astype(str)
product_data["generic_share_q"].fillna(0,inplace=True)
product_data["generic_share_r"].fillna(0,inplace=True)
product_data.rename(columns={"薬価":"prices","r_share":"shares",
                    # "elasped_0":"demand_instruments0",
                    # "elasped_1":"demand_instruments1",
                    # 'elasped_2': 'demand_instruments2',
                    # 'elasped_3': 'demand_instruments3',
                    # 'elasped_4': 'demand_instruments4',
                    # 'elasped_5': 'demand_instruments5',
                    # 'elasped_6': 'demand_instruments6',
                    # 'elasped_7': 'demand_instruments7',
                    # 'elasped_8': 'demand_instruments8',
                    # 'elasped_9': 'demand_instruments9',
                    # "year":"demand_instruments2",
                    # "generic_per":"demand_instruments4",
                    "mean_price":"demand_instruments2",
                    # "otc_dom":"demand_instruments2",
                    "lag_mean_price":"demand_instruments0",
                    # "generic_share_q":"demand_instruments7",
                    "generic_share_r":"demand_instruments1"
                    },inplace=True)
product_data=product_data.loc[product_data["shares"]>0]
# data["shares"]=data["r_share"]
product_data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,prices,後発品区分,総計,year,in_hospital,...,id_441.0,id_449.0,id_625.0,id_629.0,id_799.0,q_share,shares,generic_share_q,demand_instruments1,market_ids
0,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,292190.95,2014.0,1,...,0.0,0.0,0.0,0.0,0.0,0.000013,0.000007,0.242981,0.053692,114-2014.0
1,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,128832.00,2014.0,1,...,0.0,0.0,0.0,0.0,0.0,0.000006,0.000003,0.242981,0.053692,114-2014.0
2,114.0,解熱鎮痛消炎剤,661140020.0,インドメタシン坐剤 ５０ｍｇ,1145700J3012,19.3,0.0,106328.00,2014.0,1,...,0.0,0.0,0.0,0.0,0.0,0.000005,0.000003,0.242981,0.053692,114-2014.0
3,114.0,解熱鎮痛消炎剤,661140242.0,インテバン坐剤５０ ５０ｍｇ,1145700J3012,19.3,0.0,658257.00,2014.0,0,...,0.0,0.0,0.0,0.0,0.0,0.000029,0.000016,0.242981,0.053692,114-2014.0
4,114.0,解熱鎮痛消炎剤,661140216.0,インテバン坐剤２５ ２５ｍｇ,1145700J2016,19.3,0.0,418727.50,2014.0,0,...,0.0,0.0,0.0,0.0,0.0,0.000018,0.000010,0.242981,0.053692,114-2014.0


## random coef

In [7]:
product_data=product_data.loc[product_data["year"]==2015]
N=product_data.shape[0]
X1_formulation = pyblp.Formulation('oral+in_hospital+後発品区分',absorb='C(薬効分類)')
X2_formulation = pyblp.Formulation('-1+I(-prices)')
product_formulations = (X1_formulation, X2_formulation)
agent_formulation = pyblp.Formulation('1')
pr_integration = pyblp.Integration('product', size=5)
pr_integration


Configured to construct nodes and weights according to the level-5 Gauss-Hermite product rule with options {}.

In [8]:
agent_num=3
agent_data=pd.DataFrame(np.ones((agent_num*N,1)))
ids=product_data["market_ids"].repeat(agent_num)
ids.reset_index(drop=True,inplace=True)
agent_data["market_ids"]=ids

In [9]:
mc_integration = pyblp.Integration('monte_carlo', size=agent_num, specification_options={'seed': 0})

mc_integration
mc_problem = pyblp.Problem(product_formulations, product_data ,agent_formulation=agent_formulation,
                           agent_data=agent_data,integration=mc_integration,rc_types=["log"])
# mc_problem
# optim = pyblp.Optimization('nelder-mead',compute_gradient=False)
# optim = pyblp.Optimization('slsqp')


Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:00.

Dimensions:
 T     N     I    K1    K2    D    MD    ED 
---  -----  ---  ----  ----  ---  ----  ----
33   14348  99    3     1     1    6     1  

Formulations:
       Column Indices:            0          1         2  
-----------------------------  -------  -----------  -----
 X1: Linear Characteristics     oral    in_hospital  後発品区分
X2: Nonlinear Characteristics  -prices                    
       d: Demographics            1                       


Detected collinearity issues with [demand_instruments0, demand_instruments1, demand_instruments2] and at least one other column in ZD. Absorbed fixed effects may be creating collinearity problems. To disable collinearity checks, set options.collinear_atol = options.collinear_rtol = 0.


In [86]:
optim = pyblp.Optimization('l-bfgs-b',{'gtol': 1e-4})
optim = pyblp.Optimization('nelder-mead',compute_gradient=False)
optim = pyblp.Optimization('slsqp')
lb=-10
ub=10
# results1 = mc_problem.solve(sigma=np.eye(2),pi=[[1],[0]],sigma_bounds = [((lb, lb), (lb, lb)),
#                ((ub, ub), (ub, ub))],pi_bounds=((0,0),(10,10)),optimization=optim)
results1 = mc_problem.solve(sigma=np.eye(1),pi=1,sigma_bounds = (0,None),pi_bounds=(0,None),optimization=optim)
results1

Solving the problem ...

Nonlinear Coefficient Initial Values:
Types:  |  Sigma:      -prices     |    Pi:          1      
------  |  -------  -------------  |  -------  -------------
 Log    |  -prices  +1.000000E+00  |  -prices  +1.000000E+00

Nonlinear Coefficient Lower Bounds:
Types:  |  Sigma:      -prices     |    Pi:          1      
------  |  -------  -------------  |  -------  -------------
 Log    |  -prices  +0.000000E+00  |  -prices  +0.000000E+00

Nonlinear Coefficient Upper Bounds:
Types:  |  Sigma:      -prices     |    Pi:          1      
------  |  -------  -------------  |  -------  -------------
 Log    |  -prices      +INF       |  -prices      +INF     

Starting optimization ...



Detected that the 2SLS weighting matrix is nearly singular with condition number +6.905004E+25. To disable singularity checks, set options.singular_tol = numpy.inf.



At least one error was encountered. As long as the optimization routine does not get stuck at values of theta that give rise to errors, this is not necessarily a problem. If the errors persist or seem to be impacting the optimization results, consider setting an error punishment or following any of the other suggestions below:
The fixed point computation of delta failed to converge. This problem can sometimes be mitigated by increasing the maximum number of fixed point iterations, increasing the fixed point tolerance, choosing more reasonable initial parameter values, setting more conservative parameter or share bounds, or using different iteration or optimization configurations.

GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Projected                                
Step     Time       Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement   Gradient Norm             Theta            

Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +3.501250E+25. To disable singularity checks, set options.singular_tol = numpy.inf.



Reverted problematic elements in the Jacobian (holding beta fixed) of xi (equivalently, of delta) with respect to theta. Number of reverted elements: 28294 out of 28696.

Computed results after 00:00:58.

Problem Results Summary:
GMM     Objective      Projected    Reduced Hessian  Reduced Hessian  Clipped  Weighting Matrix
Step      Value      Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares   Condition Number
----  -------------  -------------  ---------------  ---------------  -------  ----------------
 1    +1.963272E+10  +2.696934E+07   +0.000000E+00    +0.000000E+00    5297     +4.089519E+32  

Starting optimization ...


At least one error was encountered. As long as the optimization routine does not get stuck at values of theta that give rise to errors, this is not necessarily a problem. If the errors persist or seem to be impacting the optimization results, consider setting an error punishment or following any of the other suggestions below:
Reverted problematic elemen

Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +7.233044E+25. To disable singularity checks, set options.singular_tol = numpy.inf.
Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +6.256176E+25. To disable singularity checks, set options.singular_tol = numpy.inf.


Problem Results Summary:
GMM     Objective      Projected    Reduced Hessian  Reduced Hessian  Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares   Condition Number  Condition Number 
----  -------------  -------------  ---------------  ---------------  -------  ----------------  -----------------
 2    +1.226657E-25  +7.384647E-26   +0.000000E+00    +0.000000E+00    5297     +9.880095E+30      +1.477811E+20  

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:06:29       Yes          3            26          36526       110042   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
Types:  |  Sigma:       -prices      |    Pi:           1       
------  |  -------  ---------------  |  -------  -----------

In [87]:
print(results1)

Problem Results Summary:
GMM     Objective      Projected    Reduced Hessian  Reduced Hessian  Clipped  Weighting Matrix  Covariance Matrix
Step      Value      Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares   Condition Number  Condition Number 
----  -------------  -------------  ---------------  ---------------  -------  ----------------  -----------------
 2    +1.226657E-25  +7.384647E-26   +0.000000E+00    +0.000000E+00    5297     +9.880095E+30      +1.477811E+20  

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:06:29       Yes          3            26          36526       110042   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
Types:  |  Sigma:       -prices      |    Pi:           1       
------  |  -------  ---------------  |  -------  -----------

In [88]:
# import sys
# filename = "./results/blp_0904_2015_slsqp.txt"
# with open(filename, "w") as file:
#     # Redirect the print output to the file
#     original_stdout = sys.stdout
#     sys.stdout = file
    
#     # Print the contents of the ProblemResults object
#     print(results1)
    
#     # Reset the print output back to the original stdout
#     sys.stdout = original_stdout

In [10]:
pr_integration = pyblp.Integration('product', size=5)
pr_problem = pyblp.Problem(product_formulations, product_data,agent_formulation,agent_data,integration=pr_integration,rc_types=["log"])
optim = pyblp.Optimization('bfgs',{'gtol': 1e-4})
results2 = pr_problem.solve(sigma=np.ones(1),pi=1, optimization=optim)
results2


Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:00.

Dimensions:
 T     N     I    K1    K2    D    MD    ED 
---  -----  ---  ----  ----  ---  ----  ----
33   14348  165   3     1     1    6     1  

Formulations:
       Column Indices:            0          1         2  
-----------------------------  -------  -----------  -----
 X1: Linear Characteristics     oral    in_hospital  後発品区分
X2: Nonlinear Characteristics  -prices                    
       d: Demographics            1                       
Solving the problem ...

Nonlinear Coefficient Initial Values:
Types:  |  Sigma:      -prices     |    Pi:          1      
------  |  -------  -------------  |  -------  -------------
 Log    |  -prices  +1.000000E+00  |  -prices  +1.000000E+00
Starting optimization ...



Detected collinearity issues with [demand_instruments0, demand_instruments1, demand_instruments2] and at least one other column in ZD. Absorbed fixed effects may be creating collinearity problems. To disable collinearity checks, set options.collinear_atol = options.collinear_rtol = 0.
Detected that the 2SLS weighting matrix is nearly singular with condition number +6.905004E+25. To disable singularity checks, set options.singular_tol = numpy.inf.



At least one error was encountered. As long as the optimization routine does not get stuck at values of theta that give rise to errors, this is not necessarily a problem. If the errors persist or seem to be impacting the optimization results, consider setting an error punishment or following any of the other suggestions below:
The fixed point computation of delta failed to converge. This problem can sometimes be mitigated by increasing the maximum number of fixed point iterations, increasing the fixed point tolerance, choosing more reasonable initial parameter values, setting more conservative parameter or share bounds, or using different iteration or optimization configurations.

GMM   Computation  Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Gradient                                 
Step     Time       Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement       Norm                  Theta            

Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +1.097133E+26. To disable singularity checks, set options.singular_tol = numpy.inf.



The fixed point computation of delta failed to converge. This problem can sometimes be mitigated by increasing the maximum number of fixed point iterations, increasing the fixed point tolerance, choosing more reasonable initial parameter values, setting more conservative parameter or share bounds, or using different iteration or optimization configurations.

Computed results after 00:00:58.

Problem Results Summary:
GMM     Objective      Gradient         Hessian         Hessian     Clipped  Weighting Matrix
Step      Value          Norm       Min Eigenvalue  Max Eigenvalue  Shares   Condition Number
----  -------------  -------------  --------------  --------------  -------  ----------------
 1    +7.022343E+09  +3.304886E+09  -5.502225E+17   -5.226928E+16    2177     +4.089519E+32  

Starting optimization ...


At least one error was encountered. As long as the optimization routine does not get stuck at values of theta that give rise to errors, this is not necessarily a problem. If 

Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +1.097133E+26. To disable singularity checks, set options.singular_tol = numpy.inf.
Detected that the estimated covariance matrix of aggregate GMM moments is nearly singular with condition number +1.639777E+26. To disable singularity checks, set options.singular_tol = numpy.inf.


Problem Results Summary:
GMM     Objective      Gradient         Hessian         Hessian     Clipped  Weighting Matrix  Covariance Matrix
Step      Value          Norm       Min Eigenvalue  Max Eigenvalue  Shares   Condition Number  Condition Number 
----  -------------  -------------  --------------  --------------  -------  ----------------  -----------------
 2    +1.105855E-27  +4.867475E-29  -1.316653E-22   +3.899016E-22    2177     +4.864168E+28      +1.397784E+19  

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:11:39       No           2            55         1701460      5106885  

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
Types:  |  Sigma:       -prices      |    Pi:           1       
------  |  -------  ---------------  |  -------  ---------------
 Lo